# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>План реализации проекта<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Распаковка-данных" data-toc-modified-id="Распаковка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Распаковка данных</a></span><ul class="toc-item"><li><span><a href="#Промежуточный-вывод:" data-toc-modified-id="Промежуточный-вывод:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Промежуточный вывод:</a></span></li></ul></li><li><span><a href="#Подготовка-данных-для-целей-машинного-обучения" data-toc-modified-id="Подготовка-данных-для-целей-машинного-обучения-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных для целей машинного обучения</a></span><ul class="toc-item"><li><span><a href="#Проверка-на-мультиколлинеарность" data-toc-modified-id="Проверка-на-мультиколлинеарность-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Проверка на мультиколлинеарность</a></span></li><li><span><a href="#Нормирование-признаков" data-toc-modified-id="Нормирование-признаков-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Нормирование признаков</a></span></li></ul></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение модели</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Загрузка и подготовка данных

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from scipy import stats as st

### Распаковка данных

**Описание данных**

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Пройдемся циклом по фреймам чтобы выполнить необходимые действия для первичного изучения данных:

In [3]:
datas = [geo_data_0, geo_data_1, geo_data_2]

counter = 0
for i in datas:
    print('Данные по региону №' + str(counter))
    print(i.head())
    print()
    print(i.describe())
    print()
    print(i.info())
    print()
    print('Количество дубликатов', i.duplicated().sum())
    print()
    counter += 1

Данные по региону №0
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999


#### Промежуточный вывод:

На первый взгляд данные хорошего качества. Отсутствуют пропуски и дубляжи. Все признаки приведены к оптимальным типам. Грубая, первоначальная предобработка не требуется. Посмотрим, подходят ли данные в таком виде для машинного обучения...

### Подготовка данных для целей машинного обучения

План подготовки данных:

- Проверка на мультиколлинеарность
- Нормирование данных

#### Проверка на мультиколлинеарность

Здесь тоже, в целях ускорения процесса, воспользуемся циклом:

In [4]:
for i in datas:
    print(i.corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


- Наблюдается практически полная взаимосвязь признака f2 региона №1 с целевой переменной product. Значит, возможно, предсказательная способность моделей на данных по этомурегиону будет выше.

- Есть средняя корреляция (до 0.5) между некоторыми признаками в каждом из фреймов. Это допустимо.


Увеличим признаковое пространстово каждого набора данных. Возможно, это увеличит качетво моделей:

Напишем функцию, увеличивающую признаковое пространство фрейма:

In [5]:
def feature_bulg(df): # передавать фреймы с фичами и целевым признаком
    #разобью фрейм на фичи и целевой. Целевой приклеится после раздутия признакового пространства
    data = df.iloc[:, 1:4] # можно столбцы с фичами тоже вынести в параметр функции. Пока оставлю так
    
    # генерация будущих названий столбцов
    combos = list(combinations(list(data.columns), 2))
    col_names = list(data.columns) + ['_'.join(x) for x in combos]
    
    # генерация новых признаков
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    data = poly.fit_transform(data)
    data = pd.DataFrame(data)
    data.columns = col_names
    
    data['product'] = df['product']
    
    return data

Преобразуем исходные наборы данных. Уберем сильно коррелирующие признаки.

In [6]:
reg_0_fb = feature_bulg(geo_data_0)
reg_1_fb = feature_bulg(geo_data_1)
reg_2_fb = feature_bulg(geo_data_2)

In [7]:
datas_fb = [reg_0_fb, reg_1_fb, reg_2_fb]

Чтобы было удобнее смотреть на матрицу корреляции, упростим ее, преобразовав значения корреляции в 0, если значение меньше порога, и в 1 если больше. Величину порога условно примем равной 0.9.

In [8]:
threshold = 0.9

for i in datas_fb:
    display(i.corr().applymap(lambda x: 1 if x>threshold else 0))

,f0,f1,f2,f0_f1,f0_f2,f1_f2,product
f0,1,0,0,0,0,0,0
f1,0,1,0,0,0,0,0
f2,0,0,1,0,0,0,0
f0_f1,0,0,0,1,0,0,0
f0_f2,0,0,0,0,1,0,0
f1_f2,0,0,0,0,0,1,0
product,0,0,0,0,0,0,1


,f0,f1,f2,f0_f1,f0_f2,f1_f2,product
f0,1,0,0,0,0,0,0
f1,0,1,0,0,0,0,0
f2,0,0,1,0,0,0,1
f0_f1,0,0,0,1,0,0,0
f0_f2,0,0,0,0,1,0,0
f1_f2,0,0,0,0,0,1,0
product,0,0,1,0,0,0,1


,f0,f1,f2,f0_f1,f0_f2,f1_f2,product
f0,1,0,0,0,0,0,0
f1,0,1,0,0,0,0,0
f2,0,0,1,0,0,0,0
f0_f1,0,0,0,1,0,0,0
f0_f2,0,0,0,0,1,0,0
f1_f2,0,0,0,0,0,1,0
product,0,0,0,0,0,0,1


Сильно коррелирующих между собой признаков не обнаружено.

**Промежуточный вывод:**

Данные подготовлены для нормирования. 

#### Нормирование признаков

Весь процесс нормирования сведем в функцию:

In [9]:
def normalize(data):
    # отделяем целевой признак
    features = data.drop('product', axis=1)
    target = data['product']
    
    # делим выборку на train и test, чтобы при нормализации избежать подглядывания
    features_train, features_test, target_train, target_test = train_test_split(features,
                                                                                    target,
                                                                                    test_size=0.25,
                                                                                    random_state=42)
    # запоминаем названия колонок с количественным признаком                                
    col_names = [x for x in features_train.columns]
    
    # обучаем модель нормирования на тренировочной выборке                                                                            
    scaler = StandardScaler()
    scaler.fit(features_train[col_names])
    
    # выключаем длинное предупреждение о копировании                                                                            
    pd.options.mode.chained_assignment = None

    # преобразовываем количественные признаки                                                                            
    features_train[col_names] = scaler.transform(features_train[col_names])
    features_test[col_names] = scaler.transform(features_test[col_names])  
                                                                                
    return features_train, features_test, target_train, target_test                                                                          
                                                                                

Теперь нормируем все выборки:

In [10]:
features_train_0, features_test_0, target_train_0, target_test_0 = normalize(reg_0_fb)
features_train_1, features_test_1, target_train_1, target_test_1 = normalize(reg_1_fb)
features_train_2, features_test_2, target_train_2, target_test_2 = normalize(reg_2_fb)

Проверим размеры выборок. Ничего ли потеряли в процессе преобразований:

In [11]:
_ = [features_train_0, features_test_0, target_train_0, target_test_0,
features_train_1, features_test_1, target_train_1, target_test_1,
features_train_2, features_test_2, target_train_2, target_test_2,]

for i in _:
    print(i.shape)

(75000, 6)
(25000, 6)
(75000,)
(25000,)
(75000, 6)
(25000, 6)
(75000,)
(25000,)
(75000, 6)
(25000, 6)
(75000,)
(25000,)


Судя по размерам, все данные на месте.

### Вывод:

В ходе подготовки данных, было произведено увеличение количественных признаков. Признаки были нормированы. Проведена проверка на мультиколлинеарность. 

## Обучение и проверка модели

### Обучение модели

Приступим к обучению. В условии сказано, что оптимальной моделью является модель линейной регрессии. Ее и будем обучать.

Для уменьшения количества кода напишем функцию, которая обучит модель и вернет: RMSE, среднее предсказанное значение запасов сырья в регионе и массив из предсказаний:

In [12]:
def fit_model(features_target_list):

    model = LinearRegression()
    model.fit(features_target_list[0], features_target_list[2])
    predicted = model.predict(features_target_list[1])
    rmse = np.sqrt(mean_squared_error(features_target_list[3], predicted))
    
    return rmse, predicted.mean(), predicted

Запишем в массив все необходимые для обучения фреймы:

In [13]:
lst = [[features_train_0, features_test_0, target_train_0, target_test_0],
       [features_train_1, features_test_1, target_train_1, target_test_1],
       [features_train_2, features_test_2, target_train_2, target_test_2]
      ]       

Обучим модели и выведем результат на экран:

In [14]:
counter = 0
model_predictions = []

for i in lst:
    rmse, product_mean, predicted = fit_model(i)
    model_predictions.append(predicted)
    
    print('Результаты обучения модели по данным региона №' + str(counter))
    print()
    print('RMSE = {:.2f}'.format(rmse))
    print('Средний предсказанный запас сырья в регионе = {:.2f}'.format(product_mean))

    print()
    counter += 1

Результаты обучения модели по данным региона №0

RMSE = 37.76
Средний предсказанный запас сырья в регионе = 92.40

Результаты обучения модели по данным региона №1

RMSE = 0.89
Средний предсказанный запас сырья в регионе = 68.71

Результаты обучения модели по данным региона №2

RMSE = 40.15
Средний предсказанный запас сырья в регионе = 94.77



Проверим, что массив из предсказаний успешно сохранен:

In [15]:
model_predictions

[array([101.88550056,  78.36496858, 115.36467059, ...,  82.62175506,
         81.77762923,  93.32224731]),
 array([  0.84561778,  52.9274171 , 135.11031892, ...,  26.71960811,
        109.82443197, 135.44163323]),
 array([ 98.33445209, 101.64071046,  52.81668175, ...,  63.50268623,
         83.80059303,  86.59455401])]

### Проверка модели на адекватность

Убедимся, что увеличение пространства признаков пошло модели на пользу. Обучим модель с изначальным количеством признаков и сравним результаты:

In [16]:
X_train, X_test, y_train, y_test = normalize(geo_data_0.iloc[:, 1:])
X_train.head()

,f0,f1,f2
98980,1.274786,-0.799739,-0.396677
69824,-1.600689,0.234678,-2.169283
9928,-0.323791,1.436297,1.495425
75599,0.439038,0.830679,0.185881
95621,-1.652805,0.761012,0.111734


In [17]:
rmse, product_mean, predicted = fit_model([X_train, X_test, y_train, y_test])
print('RMSE = {:.2f}'.format(rmse))
print('Средний предсказанный запас сырья в регионе = {:.2f}'.format(product_mean))

RMSE = 37.76
Средний предсказанный запас сырья в регионе = 92.40


**Промежуточный вывод:**

- RMSE не изменилась. Что говорит о том, что увеличение признакового пространства не повлияло на предсказательную способность модели.

### Вывод:

- Проведено обучение модели. RMSE колеблется в пределах 0-40. 

- Лучшей предсказательной способностью обладает модель обученная на данных по региону №1. Вследствие почти прямой взаимосвязи одной из фич с целевым признаком. 

- Проведен тест на адекватность модели, в котором модели показали практически идентичные результаты. Сделан вывод о том, что увеличение признакового пространства было излишним.

- Проведена попытка увеличения качетва модели путем избавления данных от выбросов. Первый же тест показал худшие результаты в сравнении с контрольной моделью. 

## Подготовка к расчёту прибыли

Сохраним все ключевые показатели в отдельных переменных:
- Бюджет на разработку - BUDGET
- Количетво скважин для разработки - BAREHOLS
- Доход с реализации единицы продукта - EARNINGS 

In [18]:
BUDGET = 10e6
BAREHOLS = 200
EARNINGS = 450 

Для удобства расчетов, бюджет сокращен на 3 нуля. Это поможет избежать путаницы в дальнейшем.

Определим минимальное количество запасов сырья, необходимое для безубыточной добычи:

In [19]:
min_stock_raw = BUDGET / EARNINGS / BAREHOLS
print('Минимально необходимое количество сырья в одной скважине для безубыточной разработки: {:.2f} тыс. баррелей'.\
      format(min_stock_raw))

Минимально необходимое количество сырья в одной скважине для безубыточной разработки: 111.11 тыс. баррелей


Напишем функцию вычисляющую количество сырья в 200 крупнейших скважинах региона и потенциальную прибыль от его реализации:

In [21]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 0.45 * selected.sum() / 1e3 - 10

Преобразуем список предсказаний модели в объект Series:

In [22]:
probs_0 = pd.Series(model_predictions[0], index=target_test_0.index)
probs_1 = pd.Series(model_predictions[1], index=target_test_1.index)
probs_2 = pd.Series(model_predictions[2], index=target_test_2.index)

In [23]:
probs_lst = [probs_0, probs_1, probs_2]

In [24]:
counter = 0
for i in range(len(probs_lst)):
    money = (revenue(lst[i][3], probs_lst[i], 200))
    print('Регион №' + str(counter))
    print('Потенциальная чистая прибыль от реализации сырья: {:.2f} млрд. руб.'.format(money))
    counter += 1

Регион №0
Потенциальная чистая прибыль от реализации сырья: 3.34 млрд. руб.
Регион №1
Потенциальная чистая прибыль от реализации сырья: 2.42 млрд. руб.
Регион №2
Потенциальная чистая прибыль от реализации сырья: 2.61 млрд. руб.


### Вывод:

- Среднего запаса сырья в одной скважине недостаточно для прибыльной добычи ни в одном из регионов. Нужно отбирать самые объемные месторождения. 

## Расчёт прибыли и рисков 

In [25]:
target_lst = [i[3] for i in lst]

Напишем функцию, которая выполнит процедуру Bootstrap и вернет нам фрейм с чистой прибылью по каждой из выборок:

In [26]:
def bootstrap_profit(target, predictions):
    # Инициализируем рандомайзер
    state = np.random.RandomState(12345)
    # Создаем пустой список, куда будем записывать значения чистой прибыли
    profit_list = []
    
    # Запускаем процедуру Bootstrap. Входные: количество выборок - 1000, объем выборки 500, из которых
    # отбираем 200 самых прибыльных
    for i in range(1000):
        target_500 = target.sample(n=500, replace=True, random_state=state)
        predict_500 = predictions[target_500.index]
        profit_list.append(revenue(target_500, predict_500, 200))

    profit_list = pd.Series(profit_list) #распределение прибыли
    
    return profit_list

Пройдемся циклом по данным из 3-х регионов и найдем величину риска, 95% - доверительный интервал, и среднюю прибыль:

In [27]:
count_reg = 0
for i in range(len(probs_lst)):
    # находим распределение прибыли
    profit_data = bootstrap_profit(target_lst[i], probs_lst[i])
    
    # по распределению находим 95% доверительный интервал
    confidence_interval = st.t.interval(alpha=0.95, 
                                    df=(profit_data.count() - 1), 
                                    loc=profit_data.mean(),
                                    scale=profit_data.sem())
    
    # находим среднее значение прибыли в регионе
    mean_profit = profit_data.mean()
    
    # вычисляем вероятность убытков
    
    losses = (profit_data < 0).mean() * 100
    
    heigher = profit_data.quantile(.975)
    lower = profit_data.quantile(.025)
    
    print('Регион №' + str(count_reg))
    print()
    print('95% доверительный интервал ({:.2f} : {:.2f})'.format(lower, heigher))
    print('Средняя прибыль = {:.2f} млрд. руб.'.format(mean_profit))
    print('Величина риска = {:.2f}%'.format(losses))
    print()
    count_reg += 1

Регион №0

95% доверительный интервал (-0.13 : 0.96)
Средняя прибыль = 0.44 млрд. руб.
Величина риска = 6.20%

Регион №1

95% доверительный интервал (0.06 : 0.91)
Средняя прибыль = 0.49 млрд. руб.
Величина риска = 1.10%

Регион №2

95% доверительный интервал (-0.15 : 0.95)
Средняя прибыль = 0.40 млрд. руб.
Величина риска = 7.00%



## Вывод:

- По результатам проведенной работы выяснилось, что подходящим под требования (*нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%.*) является регион №1. Другие, имеют несоответствующую величину риска. Так же, регион №1 имеет более высокий показатель средней прибыли и меньший размах доверительного интервала. Плюс ко всему, RMSE данных этого региона меньше 1, что говорит о высокой точности прогнозов. 
- Таким образом, регион для разработки это регион №1